In [1]:
import matplotlib, time, copy
import autograd.numpy as np
import autograd.scipy.stats as sps_autograd
from autograd import grad, hessian, jacobian
from statsmodels.tsa.arima_process import ArmaProcess
from scipy.optimize import minimize
from scipy.linalg import toeplitz
import pandas as pd
import scipy.stats as sps
import matplotlib.pyplot as plt 
from pymc3 import ess as effective_sample_size

The version of PyMC you are using is very outdated.

Please upgrade to the latest version of PyMC https://www.pymc.io/projects/docs/en/stable/installation.html

Also notice that PyMC3 has been renamed to PyMC.


In [2]:
"""
Simulate ARMA(1, 1) model
"""
theta0 = np.array([0.5, 0.2]) # True parameter values

# Define AR and MA coefficients
ar = np.array([1, -theta0[0]])  
ma = np.array([1, -theta0[1]])        

# Create ARMA process object
arma_process = ArmaProcess(ar, ma)

# Simulate 500 samples
N = 200
y = arma_process.generate_sample(nsample=N, scale=2) # scale is the variance of the white noise

# Define function for the whole Kalman Filter

In [3]:
def initiate_KF_parameter_values(theta, sigma2):
    """
    Initialise the parameters for the Kalman Filter
    """
    # region Initial values for Kalman Filter
    a, b = theta
    k = len(theta)

    # Simple operations that autograd can handle
    F = np.array([[a, 1.0], [0.0, 0.0]])
    G = np.array([[1.0], [-b]])
    H = np.array([[1.0, 0.0]])
    Q = np.eye(k)
    dF = np.array([
        [[1, 0], [0, 0]],  # First parameter (AR coefficient)
        [[0, 0], [0, 0]]   # Second parameter (MA coefficient)
        ])
    dF.reshape(2, 2, 2)
    dG = np.array([
        [[0], [0]],  # First parameter (AR coefficient)
        [[0], [-1.0]]   # Second parameter (MA coefficient)
        ])
    dG.reshape(2, 2, 1)

    g = np.array([1.0, a - b, a * (a - b)])

    C = np.array([
        sigma2 * (1 - 2 * a * b + b**2) / (1 - a**2),
        a * (sigma2 * (1 - 2 * a * b + b**2) / (1 - a**2)) - sigma2 * b,
        a * (a * (sigma2 * (1 - 2 * a * b + b**2) / (1 - a**2)) - sigma2 * b)
    ])

    V = np.array([
        [C[0], -b * g[0]],
        [-b * g[0], b**2 * sigma2]
    ])

    dV = np.array([
        [[(2 * sigma2 * (a-b) * (1 - a*b)) / (1 - a**2)**2, 0],
        [0, 0]],
        [[2 * sigma2 * (b-a) / (1 - a**2), -1],
        [-1, 2 * b * sigma2]]
    ])

    # Initialize the x and dx
    x = np.zeros((k, 1))
    dx = np.zeros((2, 2, 1))

    return F, G, H, dF, dG, V, dV, x, dx, k

In [4]:
def analytical_grad_hess(theta, init_sigma2, y):

    N = len(y)

    # Dictionary to store the values
    dict = {
        'analytical_grad_at_i': [],
        'log_likelihood_at_i': [],
    }

    # Initialize the values
    F, G, H, dF, dG, V, dV, x, dx, k = initiate_KF_parameter_values(theta, init_sigma2)
    sigma2_hat_sum = 0.0
    dsigma2_hat_sum = np.array([[0.0], [0.0]]).reshape(2, 1, 1)
    log_r_sum = 0.0


    # Run the Kalman filter for gradient
    for t in range(N):
        # Set the current sample size
        n = t + 1
    
        # 1. Predict
        # Predict one-step-ahead state predictive density of x_{t}
        x_predict = F @ x
        V_predict = F @ V @ F.T + G @ G.T

        # Compute forecast error and one-step-ahead predictive variance
        e_t = y[t] - (H @ x_predict)[0, 0]
        r_t = (H @ V_predict @ H.T)[0, 0]

        GdGT = np.array([G @ dG.T[0][i].reshape(1,2) for i in range(2)])

        # Kalman filter for gradient
        dx_predict = F @ dx + dF @ x
        dV_predict = F @ dV @ F.T + dF @ V @ F.T + F @ V @ dF.T + dG @ G.T + GdGT
        # dV_predict = F @ dV @ F.T + dF @ V @ F.T + F @ V @ dF.T + dG @ G.T + G @ Q @ dG.T


        # Calculate de_t and dr_t as tensor(2,1,1)
        de_t = -H @ dx_predict
        dr_t = H @ dV_predict @ H.T

        # Update sigma2 hat and gradient of sigma2 hat
        sigma2_hat_sum += e_t**2 / r_t
        sigma2_hat = sigma2_hat_sum / n
        dsigma2_hat_sum += (2 * e_t * de_t) /r_t - (e_t**2 * dr_t) / (r_t**2)
        dsigma2_hat = dsigma2_hat_sum / n

        # Update the r
        log_r_sum += np.log(r_t)

        # 2. Update
        # Kalman gain
        K = V_predict @ H.T / r_t

        # Update current state and covariance
        x = x_predict + K * e_t
        V = (np.eye(k) - K @ H) @ V_predict

        dK = (dV_predict @ H.T / r_t) - (V_predict @ H.T / r_t**2) @ dr_t
        dx = dx_predict + K @ de_t + dK * e_t
        dV = dV_predict - dK @ H @ V_predict - K @ H @ dV_predict
        
        # Compute sigma2_hat and gradient of the log-likelihood
        log_likelihood_at_i = -0.5 * (np.log(2 * np.pi) 
                                + np.log(sigma2_hat) 
                                + np.log(r_t) + e_t**2 / (r_t * sigma2_hat))  
    
        
        analytical_grad_at_i = - 0.5 * (dsigma2_hat / sigma2_hat 
                                    + dr_t / r_t 
                                    + (2 * e_t * sigma2_hat * r_t * de_t - e_t**2 * r_t * dsigma2_hat - e_t**2 * sigma2_hat * dr_t) / (sigma2_hat**2 * r_t**2)
                                    )
        

        dict['analytical_grad_at_i'].append(analytical_grad_at_i.flatten())
        dict['log_likelihood_at_i'].append(log_likelihood_at_i)

    log_likelihood = -0.5 * (N * np.log(2 * np.pi) 
                            + N * np.log(sigma2_hat) 
                            # + np.sum(np.log(r)) + N)
                            + log_r_sum + N)

    # Return the dictionary
    return dict, sigma2_hat, log_likelihood

# Function to evaluate q

In [5]:
def log_prior(theta):
    """
    Simple indepent normal prior with mean 0 and variance 10^2
    """
    return np.sum(sps_autograd.norm.logpdf(theta, 0, 10**2)) 

In [6]:
# Define the log-posterior
log_posterior = lambda theta: analytical_grad_hess(theta, init_sigma2=1.5, y=y)[2] + log_prior(theta) 
obj_func_posterior = lambda theta: -log_posterior(theta) # For maximisation purpose
# grad_obj_func_posterior = lambda theta: analytical_grad_hess(theta, init_sigma2=0.5)[0]['analytical_grad_at_i'][-1] # Gradient of obj_func_posterior
# Hess_obj_func_posterior  = hessian(obj_func_posterior) # Hessian of obj_func_posterior. Computed by automatic differentiation

# Find starting values by optimising the log-posterior
theta_optim_start = np.zeros(2)
res_optim_posterior = minimize(obj_func_posterior, theta_optim_start, method='BFGS', options={'gtol': 1e-04, 'maxiter': 1000, 'disp': True})

# Compare answers
print('True parameter values')
print(theta0)
print('MAP estimates')
MAP = res_optim_posterior.x
print(np.round(MAP, 2))

# Control variate expanded around this
thetaStar = MAP 

Optimization terminated successfully.
         Current function value: 401.065332
         Iterations: 9
         Function evaluations: 45
         Gradient evaluations: 15
True parameter values
[0.5 0.2]
MAP estimates
[0.68 0.37]


In [7]:
def initiate_control_variate_quantities(thetaStar, init_sigma2=1.5):
    """
    Creates the quantities needed to construct the second order parameter expanded Taylor control variates for the log_density.
    Output from this function will go into the function eval_q_k
    """
    N = len(y)
    p = len(thetaStar)

    # Declare the output 
    dens_at_thetaStar = np.zeros(N)
    grad_at_thetaStar = np.zeros([N, p])
    Hess_at_thetaStar = np.zeros([N, p, p])

    # Run the Kalman filter for gradient
    analytical_grad_hess_result = analytical_grad_hess(thetaStar, init_sigma2, y=y)
    dens_at_thetaStar = analytical_grad_hess_result[0]['log_likelihood_at_i']
    grad_dens = analytical_grad_hess_result[0]['analytical_grad_at_i']

    # Calculate gradient at thetaStar
    grads_all = lambda param: np.stack(
        analytical_grad_hess(param, init_sigma2=init_sigma2, y=y)[0]['analytical_grad_at_i'], axis=0
    ) 
    Hess_all_at_thetaStar = jacobian(grads_all)(thetaStar)  # (N, p, p)

    
    for t in range(N):    
        if t % 100 == 0:
            print("Processed %s observations (out of % s)" % (t, N))

        grad_at_thetaStar[t, :], Hess_at_thetaStar[t, :] = grad_dens[t], Hess_all_at_thetaStar[t]
                    
    return np.array(dens_at_thetaStar), np.array(grad_at_thetaStar), np.array(Hess_at_thetaStar)

In [ ]:
dens_at_thetaStar_ARMA, grad_at_thetaStar_ARMA, Hess_at_thetaStar_ARMA = initiate_control_variate_quantities(thetaStar)

In [ ]:
def eval_q_k(theta, dens_at_thetaStar, grad_at_thetaStar, Hess_at_thetaStar, order = 2):
    """
    Evaluates the order order parameter expanded Taylor control variates at the point theta for all observations in 
    dens_at_thetaStar, grad_at_thetaStar, Hess_at_thetaStar. Default order is 2.
    """
    const_term = dens_at_thetaStar
    if order == 0:
        q_k = const_term
    elif order == 1:
        first_term = np.sum(grad_at_thetaStar*(theta - thetaStar), axis = 1)
        q_k = const_term + first_term
    elif order == 2:
        first_term = np.sum(grad_at_thetaStar*(theta - thetaStar), axis = 1)
        second_term = 0.5*np.sum(np.sum(Hess_at_thetaStar*np.outer(theta - thetaStar, theta - thetaStar), axis = 1), axis = 1)    
        q_k = const_term + first_term + second_term
    else:
        raise ValueError("Order must be 0 <= order <= 2")
    return q_k

In [ ]:
dens_at_thetaStar, grad_at_thetaStar, Hess_at_thetaStar = dens_at_thetaStar_ARMA, grad_at_thetaStar_ARMA, Hess_at_thetaStar_ARMA

d = len(theta0) 

# Calculate gradient at thetaStar
grads_all = lambda param: np.stack(
    analytical_grad_hess(param, init_sigma2=1.5, y=y)[0]['analytical_grad_at_i'], axis=0
) 
Hess_all_at_MAP = jacobian(grads_all)(MAP)

obj_log_prior = lambda theta: -log_prior(theta)
Hess_log_prior = hessian(obj_log_prior)

# Proposal covariance
Sigma_pi = np.linalg.inv(-np.sum(Hess_all_at_MAP, axis = 0) + Hess_log_prior(MAP)) # Negative Hessian inverse evaluated at the mode
PropCov = 2.38**2/d*Sigma_pi

# Get ready for pseudo marginal Metropolis-Hastings sampling
Taylor_order = 2
N_sim = 22000 # MCMC samples
G = 10 # Number of blocks
m = 100 # Subsample size
u_c = np.random.randint(0, N, m)
u_block_indicators = np.hstack((np.repeat(np.arange(G-1), m/G), np.repeat(G-1, m - len(np.repeat(np.arange(G-1), m/G))))) # Splits the u into blocks

# Precomputed quantities for the control variates that do not depend on theta
# Following the notation in the slides
A = np.sum(dens_at_thetaStar) 
B = np.sum(grad_at_thetaStar, axis = 0)
C = np.sum(Hess_at_thetaStar, axis = 0)

if Taylor_order == 0:
    q_sum = lambda theta: A
elif Taylor_order == 1:
    q_sum = lambda theta: A + np.dot(B, theta - thetaStar)
elif Taylor_order == 2:
    q_sum = lambda theta: A + np.dot(B, theta - thetaStar) + 0.5*np.dot(theta - thetaStar, np.dot(C, theta - thetaStar))
else:
    raise NotImplementedError

# Storage
theta_init = MAP + sps.norm.rvs(0, 0.01, len(MAP))
samples = np.zeros((N_sim + 1, d))
samples[0, :] = theta_init
alphas = np.zeros(N_sim)
log_posthat_samples = np.zeros(N_sim + 1) # Keeps the estimated value of the log-posterior (up to a normalisation constant)
sigma2_lhat_samples = np.zeros(N_sim + 1) # Keeps the estimated variance of the log-likelihood estimator 

# Current parameter and evaluate quantities
theta_c = theta_init   
l_k_c = analytical_grad_hess(theta_c, init_sigma2=1.5,y=y[u_c])[2]  # log-densities at current theta, u
q_k_c = eval_q_k(theta_c, dens_at_thetaStar[u_c], grad_at_thetaStar[u_c], Hess_at_thetaStar[u_c], order = Taylor_order)  # control variates at current theta, u
log_posthat_c = q_sum(theta_c) + N*np.mean(l_k_c - q_k_c) + log_prior(theta_c)
sigma2_lhat_c = N**2/m*np.var(l_k_c - q_k_c, ddof = 1)
log_posthat_samples[0] = log_posthat_c
sigma2_lhat_samples[0] = sigma2_lhat_c

# Check
print('Estimate')
print(q_sum(theta_c) +  N*np.mean(l_k_c - q_k_c))
print('True value')
print(analytical_grad_hess(theta_c, init_sigma2=1.5,y=y)[2])

tic = time.time()
print('Subsampling MCMC for the Poisson regression model with n = %s, m = %s, G = %s and Taylor order = %s' % (N, m, G, Taylor_order))
for i in range(1, N_sim + 1):

    if i % 1000 == 0:
        print("Iteration i = {}. Acceptance prob (mean) {:.2f}. Time: {:.2f}".format(i , np.mean(alphas[:i]), time.time() - tic))

    # Propose parameter vector and subsample
    theta_p = np.random.multivariate_normal(theta_c, PropCov, size = 1).flatten() # flatten() to get 1-dim array 
    
    block_to_update = np.random.randint(0, G, 1)[0]
    u_p = copy.copy(u_c)
    indices_to_update = (u_block_indicators == block_to_update)
    u_p[indices_to_update] = np.random.randint(0, N, np.sum(indices_to_update)) 
    
    # TODO: 
    # 1. Evaluate log densities and control variates at theta_p, u_p. 
    # 2. Estimate the log-likelihood and the variance of the log-likelihood
    l_k_p = analytical_grad_hess(theta_p, init_sigma2=1.5,y=y[u_p])[2] # '?' # Your code instead of '?' # log-densities at proposed theta, u
    q_k_p = eval_q_k(theta_p, dens_at_thetaStar[u_p], grad_at_thetaStar[u_p], Hess_at_thetaStar[u_p], order = Taylor_order) # '?' # Your code instead of '?'  # control variates at proposed theta, u
    log_posthat_p = q_sum(theta_p) + N*np.mean(l_k_p - q_k_p) + log_prior(theta_p) # '?' # Your code instead of '?' # log-likelihood estimator at proposed theta, u
    sigma2_lhat_p = N**2/m*np.var(l_k_p - q_k_p, ddof = 1) # '?' # Your code instead of '?' # estimate of variance of log-likelihood estimator at proposed theta, u
    
    # log proposal densities
    log_q_p = sps.multivariate_normal.logpdf(theta_p, mean = theta_c, cov = PropCov) # log-proposal density. Symmetric (cancels), but leave for completeness           
    log_q_c = sps.multivariate_normal.logpdf(theta_c, mean = theta_p, cov = PropCov)   
    
    alpha = np.min([1, np.exp(log_posthat_p - sigma2_lhat_p/2 - log_q_p - (log_posthat_c - sigma2_lhat_c/2 - log_q_c))])
    alphas[i - 1] = alpha
    if np.random.rand() < alpha: # sample Unif(0, 1) to determine acceptance
        samples[i, :] = theta_p
        log_posthat_samples[i] = log_posthat_p 
        sigma2_lhat_samples[i] = sigma2_lhat_p
        # Proposed becomes current in next iteration
        theta_c, log_posthat_c, sigma2_lhat_c, log_q_c, u_c = theta_p, log_posthat_p, sigma2_lhat_p, log_q_p, u_p
    else:
        samples[i, :] = theta_c 
        log_posthat_samples[i] = log_posthat_c 
        sigma2_lhat_samples[i] = sigma2_lhat_c


matplotlib.rc('text', usetex = True)
fig, ax = plt.subplots()
ax.plot(log_posthat_samples)
ax.set(xlabel='MCMC iteration', ylabel= r'Estimated $log p(y|\theta)p(\theta)$' , 
       title= 'Estimated log-posterior values over MCMC iterations')

fig, ax = plt.subplots()
ax.plot(sigma2_lhat_samples, color = 'red')
ax.set(xlabel='MCMC iteration', ylabel= r'Estimated $\sigma^2_{\widehat{d}}(\theta)$' , 
       title= 'Estimated variance of log-likelihood estimator over MCMC iterations')
ax.set_yscale('log')

# Use 2000 as burn-in
samples = samples[2000:] 

# Effective sample size and inefficiency factors
ESS = np.zeros(samples.shape[1])
for j in range(samples.shape[1]):
    ESS[j] = effective_sample_size(samples[:, j])    
IF = samples.shape[0]/ESS
   
print("Mean ESS: {:.2f}. Median ESS: {:.2f}. Min ESS: {:.2f}. Max ESS: {:.2f}".format(np.mean(ESS), np.median(ESS), np.min(ESS), np.max(ESS)))
print("Mean IF : {:.2f}. Median IF : {:.2f}. Min IF : {:.2f}. Max IF : {:.2f}".format(np.mean(IF), np.median(IF), np.min(IF), np.max(IF)))


KeyboardInterrupt: 